In [2]:
import pandas as pd
import numpy as np
import warnings
# warnings.filterwarnings("ignore")

In [3]:
mails=pd.read_csv('SMS_train.csv', encoding='ISO-8859-1')

In [4]:
mails.head(6)

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
5,6,REMINDER FROM O2: To get 2.50 pounds free call...,Spam


In [125]:
non_spam_count=mails[mails['Label'] == 'Non-Spam'].count()
non_spam_count

S. No.          835
Message_body    835
Label           835
num_urls        835
num_emails      835
num_num         835
dtype: int64

In [58]:
mails.shape

(957, 3)

In [59]:
mails.isnull().sum()

S. No.          0
Message_body    0
Label           0
dtype: int64

In [60]:
mails.duplicated().sum()

np.int64(0)

In [61]:
for row in mails['Message_body'].head(4):
    print('\n')
    print(row)



Rofl. Its true to its name


The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free


Pity, * was in mood for that. So...any other suggestions?


Will ü b going to esplanade fr home?


In [99]:
import re
pattern=r"http\S+|www\S+"
mails['Message_body']=mails['Message_body'].str.replace(pattern, '<URL>', regex=True)
mails['num_urls'] = mails['Message_body'].str.count(r"<URL>")
mails_with_urls = mails[mails['num_urls'] > 0]
print(mails_with_urls['Message_body'])

73     SMS SERVICES for your inclusive text credits p...
104    <URL> widelivecomindex wmlid<NUM>ad<NUM>a<NUM>...
108    Txt CALL to No <NUM>  claim your reward of <NU...
114    For the most sparkling shopping breaks from <N...
192    Free entry to the gr<NUM>prizes wkly comp <NUM...
275    XMAS iscoming  ur awarded either <NUM> CD gift...
284    URGENT! You have won a <NUM> week FREE members...
300    If you dont your prize will go to another cust...
354    Natalja <NUM>F is inviting you to be her frien...
368    Call Germany for only <NUM> pence per minute! ...
427    FREE for <NUM>st week! No<NUM> Nokia tone <NUM...
453    Get <NUM> Lions England tone reply lionm <NUM>...
526    Natalie <NUM>F is inviting you to be her frien...
563    CDs <NUM>u Congratulations ur awarded <NUM> of...
582    goldviking <NUM>M is inviting you to be his fr...
602    Dear Voucher holder Have your next meal on us ...
690    lyricalladie<NUM>F is inviting you to be her f...
733    New Mobiles from <NUM> M

In [95]:
import re
pattern = r"\S+@\S+"
mails['Message_body']=mails['Message_body'].str.replace(pattern, '<EMAIL>', regex=True)
mails['num_emails']=mails['Message_body'].str.count(r'<EMAIL>')
mails['num_emails'].sum()

np.int64(3)

In [96]:
pattern_num=r"([0-9]+)"
mails['Message_body']=mails['Message_body'].str.replace(pattern_num, '<NUM>',  regex=True)
mails['num_num']=mails['Message_body'].str.count(r'<NUM>')
mails['num_num'].sum()


np.int64(753)

In [101]:
pattern=r"[^\w\s!$%<>]"
mails['Message_body']=mails['Message_body'].str.replace(pattern, "", regex=True)

In [271]:
from sklearn.feature_extraction.text import CountVectorizer

vec=CountVectorizer()
text=vec.fit_transform(mails['Message_body'])

y=mails['Label']
text.shape

(957, 2991)

In [272]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1)

In [273]:
from sklearn.naive_bayes import MultinomialNB

clf_nb=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [274]:
from sklearn.model_selection import cross_val_score
valid=cross_val_score(estimator=clf_nb, X=text, y=y, cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[1.         0.96875    0.96875    1.         0.95833333 0.94791667
 0.95833333 0.96842105 0.95789474 0.93684211]
Mean for cross val is 0.9665241228070176
std for the cross val is 0.019211142769288883


In [275]:
clf_nb.fit(x_train,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [276]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
train_pred=clf_nb.predict(x_train)
print(f"Classificatin Report\n {classification_report(y_train, train_pred)}")
print('\n\n')
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred)}")
print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred)}")

Classificatin Report
               precision    recall  f1-score   support

    Non-Spam       1.00      1.00      1.00       667
        Spam       0.97      0.98      0.97        98

    accuracy                           0.99       765
   macro avg       0.98      0.99      0.99       765
weighted avg       0.99      0.99      0.99       765




Confudion Matrix  
 [[664   3]
 [  2  96]]



Accuracy score   0.9934640522875817


In [277]:
#checking score based on set test
pred = clf_nb.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.99      0.99      0.99       168
        Spam       0.92      0.92      0.92        24

    accuracy                           0.98       192
   macro avg       0.95      0.95      0.95       192
weighted avg       0.98      0.98      0.98       192




Confudion Matrix  
 [[166   2]
 [  2  22]]



Accuracy score   0.9791666666666666


In [278]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(
    n_estimators=90,          # больше деревьев
    max_depth=10,              # ограничиваем глубину, чтобы избежать переобучения
    
    max_features="sqrt",       # число признаков на каждом разбиении
    class_weight="balanced",   # учитываем дисбаланс классов
    random_state=42            # фиксируем случайность
)

In [279]:
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf_rf , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.97916667 0.9375     1.         1.         0.96875    0.96875
 0.95833333 0.98947368 0.93684211 0.97894737]
Mean for cross val is 0.9717763157894739
std for the cross val is 0.02146319331512066


In [280]:
clf_rf.fit(x_train ,y_train)

,n_estimators,90
,criterion,'gini'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [281]:
train_pred2 = clf_rf.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred2)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred2)}")


print(f"Accuracy score   {accuracy_score(y_train ,train_pred2)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.99      1.00      1.00       667
        Spam       1.00      0.94      0.97        98

    accuracy                           0.99       765
   macro avg       1.00      0.97      0.98       765
weighted avg       0.99      0.99      0.99       765




Confudion Matrix  
 [[667   0]
 [  6  92]]
Accuracy score   0.9921568627450981


In [282]:
pred1 = clf_rf.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred1)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred1)}")

print("\n")
print(f"Accuracy score   {accuracy_score(y_test , pred1)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.96      0.99      0.98       168
        Spam       0.94      0.71      0.81        24

    accuracy                           0.96       192
   macro avg       0.95      0.85      0.89       192
weighted avg       0.96      0.96      0.96       192




Confudion Matrix  
 [[167   1]
 [  7  17]]


Accuracy score   0.9583333333333334


In [283]:
from xgboost import XGBClassifier
clf_xg = XGBClassifier()

In [284]:
from sklearn.model_selection import cross_val_score
y=y.map({"Non-Spam":0, "Spam":1})
valid = cross_val_score(estimator = clf_xg , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.98958333 0.9375     0.98958333 1.         0.95833333 0.97916667
 0.95833333 0.97894737 0.97894737 0.96842105]
Mean for cross val is 0.9738815789473685
std for the cross val is 0.017582523185705443


In [286]:
y_train=y_train.map({"Non-Spam":0, "Spam":1})
clf_xg.fit(x_train , y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [287]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred3 = clf_xg.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      1.00      1.00       667
           1       1.00      0.99      0.99        98

    accuracy                           1.00       765
   macro avg       1.00      0.99      1.00       765
weighted avg       1.00      1.00      1.00       765




Confudion Matrix  
 [[667   0]
 [  1  97]]



Accuracy score   0.9986928104575163


In [288]:
pred3 = clf_xg.predict(x_test)
y_test=y_test.map({"Non-Spam":0, "Spam":1})
print(f"Classification Repport\n  {classification_report(y_test , pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.98      0.98      0.98       168
           1       0.87      0.83      0.85        24

    accuracy                           0.96       192
   macro avg       0.92      0.91      0.92       192
weighted avg       0.96      0.96      0.96       192




Confudion Matrix  
 [[165   3]
 [  4  20]]



Accuracy score   0.9635416666666666


In [325]:
from catboost import CatBoostClassifier

clf_cb = CatBoostClassifier(
    iterations=120,
    learning_rate=0.05,
    depth=5,
    l2_leaf_reg=7,
    loss_function='Logloss',
    eval_metric='F1',         
    class_weights=[1, 4],   
    random_seed=42,
    verbose=50,
    early_stopping_rounds=50
)

In [326]:
from sklearn.model_selection import cross_val_score

valid = cross_val_score(estimator = clf_cb , X =text , y=y , cv=10, scoring='f1')
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

0:	learn: 0.9032258	total: 12.1ms	remaining: 1.44s
50:	learn: 0.9351852	total: 555ms	remaining: 750ms
100:	learn: 0.9797753	total: 1.17s	remaining: 219ms
119:	learn: 0.9854423	total: 1.38s	remaining: 0us
0:	learn: 0.9181818	total: 7.46ms	remaining: 887ms
50:	learn: 0.9449541	total: 566ms	remaining: 766ms
100:	learn: 0.9821429	total: 1.15s	remaining: 216ms
119:	learn: 0.9832402	total: 1.35s	remaining: 0us
0:	learn: 0.9001148	total: 9.98ms	remaining: 1.19s
50:	learn: 0.9313155	total: 566ms	remaining: 766ms
100:	learn: 0.9797753	total: 1.1s	remaining: 208ms
119:	learn: 0.9854423	total: 1.31s	remaining: 0us
0:	learn: 0.9032258	total: 12.4ms	remaining: 1.47s
50:	learn: 0.9280742	total: 592ms	remaining: 801ms
100:	learn: 0.9743017	total: 1.15s	remaining: 216ms
119:	learn: 0.9810479	total: 1.35s	remaining: 0us
0:	learn: 0.9285714	total: 10.6ms	remaining: 1.26s
50:	learn: 0.9411765	total: 552ms	remaining: 747ms
100:	learn: 0.9854423	total: 1.11s	remaining: 210ms
119:	learn: 0.9865471	total: 1.

In [327]:
clf_cb.fit(x_train , y_train)

0:	learn: 0.9083871	total: 11.4ms	remaining: 1.35s
50:	learn: 0.9307190	total: 518ms	remaining: 701ms
100:	learn: 0.9836888	total: 1s	remaining: 189ms
119:	learn: 0.9836888	total: 1.18s	remaining: 0us


In [328]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred4 = clf_cb.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred4)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred4)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred4)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      0.98      0.99       667
           1       0.88      1.00      0.94        98

    accuracy                           0.98       765
   macro avg       0.94      0.99      0.96       765
weighted avg       0.98      0.98      0.98       765




Confudion Matrix  
 [[654  13]
 [  0  98]]



Accuracy score   0.9830065359477124


In [329]:
pred4 = clf_cb.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred4)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred4)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred4)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.99      0.98      0.99       168
           1       0.88      0.92      0.90        24

    accuracy                           0.97       192
   macro avg       0.93      0.95      0.94       192
weighted avg       0.97      0.97      0.97       192




Confudion Matrix  
 [[165   3]
 [  2  22]]



Accuracy score   0.9739583333333334


In [337]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim

vec = CountVectorizer()  
X = vec.fit_transform(mails['Message_body']).toarray()
# print(X[0].shape)
le = LabelEncoder()
y = le.fit_transform(mails['Label']) 

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1) 
dataset = TensorDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

class SpamClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear1=nn.Linear(input_dim, 128)
        self.relu1=nn.ReLU()
        self.linear2=nn.Linear(128, 64)
        self.relu2=nn.ReLU()
        self.linear3=nn.Linear(64, 1) 
       
    def forward(self, x):
        x=self.linear1(x)
        x=self.relu1(x)
        x=self.linear2(x)
        x=self.relu2(x)
        x=self.linear3(x)
        return x

input_dim=X_tensor.shape[1]
model=SpamClassifier(input_dim)
loss_function=nn.BCEWithLogitsLoss() 
optimizer=optim.Adam(model.parameters(), lr=0.001)

epochs=10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        preds = torch.sigmoid(outputs)
        preds = (preds > 0.5).float()
        all_preds.append(preds)
        all_labels.append(batch_y)

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

accuracy = (all_preds == all_labels).float().mean()
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 0.4823
Epoch 2/10, Loss: 0.0833
Epoch 3/10, Loss: 0.0123
Epoch 4/10, Loss: 0.0044
Epoch 5/10, Loss: 0.0023
Epoch 6/10, Loss: 0.0015
Epoch 7/10, Loss: 0.0010
Epoch 8/10, Loss: 0.0007
Epoch 9/10, Loss: 0.0006
Epoch 10/10, Loss: 0.0004
Test Accuracy: 0.9896
